In [57]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from pyspark.sql import SparkSession,SparkContext
from pyspark.sql import functions as sf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cannot import name 'SparkContext'
Traceback (most recent call last):
ImportError: cannot import name 'SparkContext'



In [48]:
spark = SparkSession \
    .builder \
    .appName("Spark Test") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
data_matched_path = "s3://multinli/MNLI/dev_matched.tsv"
data_test_path = "s3://multinli/MNLI/test_matched.tsv"

stoplist = "s3://multinli/utils/stopwords.txt"

# path
# s3://multinli/MNLI/dev_matched.tsv
# s3://multinli/MNLI/test_matched.tsv  
# s3://multinli/MNLI/dev_mismatched.tsv
# s3://multinli/MNLI/test_mismatched.tsv

cols=['genre','sentence1','sentence2']

data_matched = spark.read.csv(data_matched_path,header=True,sep='\t',mode='w')

data_test_matched = spark.read.csv(data_test_path,header=True,sep='\t',mode='DROPMALFORMED')


getData = data_matched.select(cols)
getTestData = data_test_matched.select(cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
getData.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+
|     genre|           sentence1|           sentence2|
+----------+--------------------+--------------------+
|     slate|The new rights ar...|Everyone really l...|
|government|This site include...|The Government Ex...|
| telephone|uh i don't know i...|I like him for th...|
| telephone|yeah i i think my...|My favorite resta...|
| telephone|i don't know um d...|     I know exactly.|
+----------+--------------------+--------------------+
only showing top 5 rows

In [51]:
getTestData.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+
|     genre|           sentence1|           sentence2|
+----------+--------------------+--------------------+
|    travel|Hierbas, ans seco...|Hierbas is a name...|
|government|The extent of the...|Many people would...|
|government|Timely access to ...|It is in everyone...|
|    travel|Based in the Auve...|The French govern...|
|    travel|Built in 1870, it...|It was constructe...|
|     slate|95 a year and you...|The cost was 95 a...|
|government|Eighteen were web...|Many businesses d...|
|government|Those who do not ...|Those who do not ...|
| telephone|it's an accusatio...|They are behaving...|
|government|However, one can ...|It is possible so...|
+----------+--------------------+--------------------+
only showing top 10 rows

In [58]:
concatSentence = getData.withColumn('joined_sentence', sf.concat(sf.col('sentence1'),sf.lit(' '),\
                                                        sf.col('sentence2'))).cache()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
concatSentence.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+--------------------+
|     genre|           sentence1|           sentence2|     joined_sentence|
+----------+--------------------+--------------------+--------------------+
|     slate|The new rights ar...|Everyone really l...|The new rights ar...|
|government|This site include...|The Government Ex...|This site include...|
| telephone|uh i don't know i...|I like him for th...|uh i don't know i...|
| telephone|yeah i i think my...|My favorite resta...|yeah i i think my...|
| telephone|i don't know um d...|     I know exactly.|i don't know um d...|
+----------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [70]:
data_matched.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9815

In [74]:
import string, nltk
from nltk.corpus import stopwords

def word_tokenize1(record):
    try:
        gerne,senten1, senten2,joined_senten = record
        lowerW = joined_senten.lower()
        return nltk.word_tokenize(joined_senten)
    except:
        print(joined_senten)
        return ()

    
    
stop_words=set(stopwords.words('english'))
list_punct=list(string.punctuation)

#lower each letter
words = concatSentence.rdd.flatMap(word_tokenize1)

# words.show(5)
# #remove some stop words
stopW = words.filter(lambda word : word[0] not in stop_words and word[0] != '')
# #remove punctuation in sentence
filtered_data = stopW.filter(lambda punct : punct not in list_punct)




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['The', 'new', 'rights', 'nice', 'enough', 'Everyone', 'really', 'likes', 'newest', 'benefits', 'This', 'list', 'winners', 'Government', 'Executive', 'The', 'Government', 'Executive', 'housed', 'website', 'not', 'be', 'uh', "n't", 'know', 'have', 'emotions', 'him', 'uh', 'like', 'him', 'but', 'love', 'beat', 'him', 'I', 'like', 'him', 'for', 'part', 'but', 'would', 'enjoy', 'beat', 'him', 'favorite', 'restaurant', 'been', 'closest', 'know', 'closest', 'long', "'s", 'criteria', 'know', 'good', 'food', 'My', 'favorite', 'restaurants', 'least', 'hundred', 'from', 'house', "n't", 'know', 'um', 'lot', 'camping', 'I', 'know', 'exactly', 'well', 'would', 'be', 'help', 'wish', 'would', 'here', 'we', 'have', 'got', 'little', 'landfill', 'left', 'we', "'re", 'going', 'run', 'before', 'end', "'s", 'really', 'going', 'be', 'We', 'have', 'plenty', 'landfill', 'know', 'college', 'worked', 'I', 'college', 'but', 'never', 'worked', 'Calcutta', 'be', 'production', 'center', 'having', 'pretensions', 'cr

In [76]:
print(list_punct)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']

In [89]:
import pandas as pd

stopWord = pd.read_txt(stoplist)

len(stopWord)

# stopW = words.filter(lambda word : word[0] not in stopWords and word[0] != '')

# stopW.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'

